In [49]:
class BindParam(object):
    
    def __init__(self, param, key):
        self.param = param
        self.key = key

    def get(self):
        return self.param[self.key]
    
    def set(self, value):
        self.param[self.key] = value
    def __str__(self):
        return f"BindParam({self.param.param_name}.{self.key}={self.get()})"

In [50]:
"""

20221028


"""
import copy,sys
from collections  import  OrderedDict

class Param(object):
    def __init__(self, param_name="param", **kargs):
        self._param_name = param_name
        self.regist_from_dict(kargs)


    def regist_from_dict(self, _dict):
        assert isinstance(_dict, dict)
        for key, val in _dict.items():
            self.check_key(key)
            self.set(key, val)
        return self

    def regist_child(self, param_name: str, init_param=None):
        self[param_name] = init_param.clone() if init_param is not None else Param()
        return self[param_name]

    @property
    def name(self):
        name = self._param_name.split(".")[-1]
        return name

    @property
    def param_name(self):
        return self._param_name

    def check_key(self, key):
        assert (key != "param_name")
        assert (key != "name")
        assert (key != "keys")
        assert (key != "vals")
        assert (key != "items")

    def update_name(self, last_name, key):
        self._param_name = last_name + "." + key
        for key, val in self.__dict__.items():
            if isinstance(val, Param):
                val.update_name(self._param_name, key)


    # 功能 A["a"]
    def __setitem__(self, key, value):
        super(Param, self).__setattr__(key, value)
        if isinstance(value, Param):
            value.update_name(self._param_name, key)

    # self.__dict__[key] = value
    def __getitem__(self, attr):
        # print(attr)
        return self.__getattribute__(attr)


    def __delitem__(self, key):
        try:
            del self.__dict__[key]
        except KeyError as k:
            return None

    # 功能  A.a
    def __setattr__(self, key, value):
        if hasattr(self,key) and isinstance(self[key],BindParam):
            self[key].set(value)
            print("set bind param {}".format(key))
        else:
            super(Param, self).__setattr__(key, value)
            if isinstance(value, Param):
                value.update_name(self._param_name, key)

        
    def __getattribute__(self, attr):
        # print(attr)
        value=super(Param, self).__getattribute__(attr)
        if isinstance(value,BindParam):
            value=value.get()
        return value


    # 	"""|
    # 	重载此函数防止属性不存在时__getattribute__报错，而是返回None
    # 	那“_ getattribute_”与“_ getattr_”的最大差异在于：
    # 	1. 无论调用对象的什么属性，包括不存在的属性，都会首先调用“_ getattribute_”方法；
    # 	2. 只有找不到对象的属性时，才会调用“_ getattr_”方法；
    # 	:param attr:
    # 	:return:
    # 	"""
    # 	return super(Param, self).__getattr__(attr)
    # raise Exception("attr:[{}] is not existing".format(attr))
    def __delattr__(self, key):
        try:
            del self.__dict__[key]
        except KeyError as k:
            return None

    # def __str__(self):
    # 	string=""
    # 	for key,val in self.__dict__.items():
    # 		if key is "_name": continue
    # 		if isinstance(val,Param):
    # 			string += self._name + "{}=Param()\n".format(key)
    # 			string +="{}".format(val)
    # 		else:
    # 			string +=self._name+"{}={}\n".format(key,val)
    # 	return string
    def __str__(self):
        string = self._param_name + "=Param()\n"
        for key, val in self.__dict__.items():
            if key == "_param_name": continue
            if isinstance(val, Param):
                string += str(val)
            elif isinstance(val, str):
                string += self._param_name + ".{}='{}'\n".format(key, val)

            else:
                string += self._param_name + ".{}={}\n".format(key, val)
        return string

    def __len__(self):
        return len(self.keys())

    def keys(self):
        keys = [key for key in self.__dict__.keys() if key != "_param_name"]
        return keys

    def values(self):
        return [self[key] for key in self.keys()]

    def items(self):
        return [item for item in self if item[0] in self.keys()]


    def get(self, k, v, separator="."):
        if separator not in k:
            return self[k]
        else:
            _param=self
            pointers= k.split(separator)
            for _k in  pointers[:-1]:
                _param=_param[_k]
                assert(isinstance(_param,Param))
            return _param[pointers[-1]]

    def set(self, k, v, separator="."):
        if separator not in k:
            self[k]=v
        else:
            _param=self
            pointers= k.split(separator)
            for _k in  pointers[:-1]:
                _param=_param[_k]
                assert(isinstance(_param,Param))
            _param[pointers[-1]]=v
    def bind(self,param,key):
        self[key]=BindParam(param,key)

    def hasattr(self,k,separator="."):
        if separator not in k:
            return hasattr(self,k)
        else:
            cul_key=k.split(separator)[0]
            if not hasattr(self,cul_key):
                return False
            else:
                child_param = self[cul_key]
                child_key = k[len(cul_key) + 1:]
                assert (isinstance(child_param, Param))
                return child_param.hasattr(child_key)

    def clone(self):
        return copy.deepcopy(self)


In [55]:

base_param = Param()
base_param.repeat=1

DATA_ROOT="/media/lyushuai/Data/DATASET/multi_garde_2/"

mu=[0,0.333,0.666,1]
# sigma=[0.01]+[0.035]*3
base_param.model=Param(option="OrdNetwork")
sigma=0.035
base_param.task="train"

teatch_model="./OUTPUT/base_param,model.option-OrdNetwork,model.OrdNetwork.stds-0.05/checkpoint/OrdNetwork_epo0150.pth"
base_param.model.input_size=(512,512)
base_param.model.OrdNetwork=Param(out_layer=3,means=mu, stds=sigma,optim_style="log_prob",tau=0.1)
base_param.model.GradeClassNetwork=Param(out_layer=3,tau=0.3, alpha=10,optim_style="consim_mse")

base_param.model.DistillModel=Param(out_layer=2,means=mu, stds=sigma,optim_style="log_prob",
                                       distill_weight=1,gls_loss_weight=1,distill_method="mse",
        t_model_path=teatch_model)
base_param.model.DistillModel.distill_weight=1
base_param.model.MultiClassNet=Param(out_layer=len(mu))

base_param.model.CplNet=Param(model_name="R18_512_SPE",num_ranks=4,tau=0.11,loss_lam=5.0)
#
# base_param.gls_loss=ET.Param(means=mu, stds=sigma,
#                              optim_style="noise_label")
base_param.data=Param()
base_param.data.root=DATA_ROOT
base_param.data.config_name="train_vaild_ratio_82_G4C3"
base_param.data.img_size=(128,128)
base_param.data.means=[0.485, 0.456, 0.406]
base_param.data.stds=[0.229, 0.224, 0.225]


base_param.data.bind(base_param.model,"input_size")

base_param.data.input_size1= BindParam(base_param.model,"input_size")

In [52]:
print(base_param.data["input_size"])

(512, 512)


In [56]:
print(base_param.data.input_size1)

(512, 512)


In [57]:
base_param.model.input_size=1
print(base_param.data.input_size1)

1


In [58]:
parm=base_param.clone()

In [59]:
parm.model.input_size=2
print(parm.data.input_size1)

2


In [60]:
print(base_param.data.input_size1)

1


In [61]:
print(base_param.data.input_size)

1
